# Web Scraping del sito Subito.it



## Import delle librerie


In [1]:
import requests
import bs4
import csv
from tqdm import tqdm_notebook as tqdm
import json
import pprint
import re

## Definizione url di riferimento

Per semplificare il progetto la ricerca riguarderà tutti gli annunci trovati filtrando per regione Lombardia e prodotto iPhone.


In [2]:
webpage = f"https://www.subito.it/annunci-lombardia/vendita/usato/?q=iPhone&o=1"

Cerco tutti gli annunci nella pagina:

In [4]:
response = requests.get(webpage)
doc = bs4.BeautifulSoup(response.text, "lxml")
annunci = doc.find_all('a', class_="AdElements__Item--link-L2hvbWUv")

In [5]:
len(annunci)

33

Ci sono 33 annunci nella pagina!

Test del codice per lo scarico delle prime informazioni nella pagina di ricerca degli annunci:

In [6]:
annunci_list = []
for annuncio in annunci:
    titolo = annuncio.find("h2").text.replace("\n", "").strip()
    annuncio_id = annuncio["href"]
    url = annuncio["href"]
    try:
        prezzo = annuncio.find("div",class_ = "AdElements__ItemPrice--container-L2hvbWUv").text.replace("\n", "").replace("€","").strip()
    except:
        prezzo = "N/A"
    try:
        data_paese = annuncio.select(" div.AdElements__Item--dateLocation-L2hvbWUv > div > span")[0].text
    except:
        data_paese = "N/A"
    annunci_list.append({'id': annuncio_id, 'titolo': titolo, 'link': url, 
                         'prezzo': prezzo, 'datapaese': data_paese})
    
print(annunci_list[0])

{'id': 'https://www.subito.it/telefonia/iphone-7-brescia-300878677.htm', 'titolo': 'IPhone 7', 'link': 'https://www.subito.it/telefonia/iphone-7-brescia-300878677.htm', 'prezzo': '275', 'datapaese': 'Oggi alle 10:05 - Brescia (BS)'}


Ok, ora posso definire la funzione per il parsing generico degli annunci:

In [7]:

def parse_annuncio(annuncio):
    titolo = annuncio.find("h2").text.replace("\n", "").strip()
    url = annuncio["href"]
    annuncio_id = url.split('-')[-1].split('.')[0]
    try:
        prezzo = annuncio.find("div",class_ = "AdElements__ItemPrice--container-L2hvbWUv").text.replace("\n", "").replace("€","").strip()
    except:
        prezzo = "N/A"
    try:
        data_paese = annuncio.select(" div.AdElements__Item--dateLocation-L2hvbWUv > div > span")[0].text
    except:
        data_paese = "N/A"
    
    return {'id': annuncio_id, 'titolo': titolo, 'link': url, 
                         'prezzo': prezzo, 'datapaese': data_paese}


In un ciclo for gestisco il download degli annunci per ogni pagina:

In [9]:
annunci_list = []
for num in tqdm(range(1,11)):
    webpage = f"https://www.subito.it/annunci-lombardia/vendita/usato/?q=iPhone&o={num}"
    response = requests.get(webpage)
    doc = bs4.BeautifulSoup(response.text, "lxml")
    annunci = doc.find_all('a', class_="AdElements__Item--link-L2hvbWUv")
    for annuncio in annunci:
        annunci_list.append(parse_annuncio(annuncio))

print(len(annunci_list))
        

A Jupyter Widget


33


## Come terminare lo scraping?

Problema: *quando mi fermo?*
- cerco all'interno della pagina il numero di annuncio totali e continuo finco a che il numero degli annunci letti non è uguale al totale


In [11]:
firstpage = f"https://www.subito.it/annunci-lombardia/vendita/usato/?q=iPhone&o=1"
firstresponse = requests.get(firstpage)
firstdoc = bs4.BeautifulSoup(firstresponse.text, "lxml")
adscount = int(firstdoc.find_all("div", class_="ads-count")[0].select("span")[0].text.replace('.',''))

5498


In [13]:
annunci_list = []
annunci_letti = 0
num = 0
while annunci_letti < adscount:
    num = num + 1
    webpage = f"https://www.subito.it/annunci-lombardia/vendita/usato/?q=iPhone&o={num}"
    response = requests.get(webpage)
    doc = bs4.BeautifulSoup(response.text, "lxml")
    annunci = doc.find_all('a', class_="AdElements__Item--link-L2hvbWUv")
    for annuncio in annunci:
        try:
            annunci_list.append(parse_annuncio(annuncio))
        except:
            pass
    annunci_letti = len(annunci_list)

## Pandas
Utilizzo la libreria Pandas per salvare i risultati in un file csv:

In [15]:
import pandas as pd
ds_annunci = pd.DataFrame(annunci_list)
ds_annunci.set_index("id")
ds_annunci.head()

,datapaese,id,link,prezzo,titolo
0,Oggi alle 10:05 - Brescia (BS),300878677,https://www.subito.it/telefonia/iphone-7-bresc...,275,IPhone 7
1,Oggi alle 10:00 - Romano di Lombardia (BG),300877611,https://www.subito.it/telefonia/iphone-8-256-g...,460,"IPhone 8, 256 gb"
2,Oggi alle 09:59 - Varese (VA),300877472,https://www.subito.it/telefonia/iphone-6s-gold...,200,IPhone 6s Gold
3,Oggi alle 09:58 - Busto Arsizio (VA),300877282,https://www.subito.it/telefonia/iphone-7-plus-...,350,IPhone 7 Plus 128gb
4,Oggi alle 09:56 - Legnano (MI),300876918,https://www.subito.it/telefonia/iphone-6s-32gb...,229,IPhone 6S 32GB di 2 giorni


In [16]:
ds_annunci.to_csv("./subito_it_annunci.csv")

## Le pagine degli annunci ###
Ora l'obiettivo è scaricare i dettagli dei singoli annunci e le foto.
# Geocoding
Per avere una maggiore precisione riguardo all'informazione geografica degli annunci, richiamo il servizio di geocoding in modo da ottenere una geolocalizzazione più strutturata.


In [17]:
import json
# apro il file csv
import pandas as pd
ds_annunci = pd.read_csv("./subito_it_annunci.csv", index_col="id")
ds_annunci.head()

,Unnamed: 0,datapaese,link,prezzo,titolo
id,,,,,
300878677,0,Oggi alle 10:05 - Brescia (BS),https://www.subito.it/telefonia/iphone-7-bresc...,275.0,IPhone 7
300877611,1,Oggi alle 10:00 - Romano di Lombardia (BG),https://www.subito.it/telefonia/iphone-8-256-g...,460.0,"IPhone 8, 256 gb"
300877472,2,Oggi alle 09:59 - Varese (VA),https://www.subito.it/telefonia/iphone-6s-gold...,200.0,IPhone 6s Gold
300877282,3,Oggi alle 09:58 - Busto Arsizio (VA),https://www.subito.it/telefonia/iphone-7-plus-...,350.0,IPhone 7 Plus 128gb
300876918,4,Oggi alle 09:56 - Legnano (MI),https://www.subito.it/telefonia/iphone-6s-32gb...,229.0,IPhone 6S 32GB di 2 giorni


Per ogni annuncio vado a richiamare il link e a procedere con lo scarico dalla pagina di dettaglio:

In [21]:
dettagli = []
for annuncio_id, annuncio in tqdm(ds_annunci.iterrows(), total=ds_annunci.shape[0]):
    dettagli_row = {}
    link = annuncio["link"]
    response = requests.get(link)
    doc = bs4.BeautifulSoup(response.text, "lxml")
    try:
        descrizione = doc.find("div", class_="description").text.replace('\n', ' ').strip()
        regex = "([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)"
        p = re.findall(regex, descrizione)
        if len(p) > 0:
            email = p[0] # prendo la prima per semplicità
        else:
            email = ''
    except:
        continue
    try:
        nome_venditore = doc.select("div.publisher_info > h2")[0].text.replace('\n', ' ').strip()
    except:
        nome_venditore = "N/A"
    try: 
        phone = doc.select("div#contact-actions-container")[0]["data-prop-phone"]
    except:
        phone = 'N/A'
    try: 
        userId = doc.select("div#user_info_container")[0]["data-user-id"]
    except:
        userId = 'N/A'
    id_ad = doc.select("main")[0]["data-id"]
    date_ad = doc.select("time")[0]["datetime"]
    dettagli_row.update({'id': id_ad, 'descrizione': descrizione,  'nome_venditore': nome_venditore,
                        'phone': phone, 'date_ad': date_ad, 'userId':userId, 'email' : email})
    table = doc.select("div.summary > table ")[0]
    new_table = pd.DataFrame(columns=range(0,2), index = [0]) 
    row_marker = 0
    for row in table.find_all('tr'):
        columns = row.find_all('td')
        label = columns[0].text.replace('\n', ' ').strip()
        value = columns[1].text.replace('\n', ' ').strip()
        dettagli_row.update({label:value})
    ## geocoding
    lat = ""
    lon = ""
    try:
        key = "T4eqDjtnpzWsfeMBZgKAqKobvcICurpU"
        geocode_url = f"http://www.mapquestapi.com/geocoding/v1/address?key={key}&location={dettagli_row.get('Comune').split(' ')[0]}"
        response = requests.get(geocode_url)
        geo = json.loads(response.text)
        lat = geo['results'][0]['locations'][0]['latLng']['lat']
        lon = geo['results'][0]['locations'][0]['latLng']['lng']
    except:
        pass
    dettagli_row.update({"lat": lat, "lon": lon})
    #scarico le immagini
    imgs = doc.select("img")
    i = 0
    for img in imgs:
        src = img['src']
        if src.find("logo_print.png") < 0:
            img_file = requests.get(src, stream=True)
            if img_file.status_code == 200:
                with open("./img_subito/img_" + str(id_ad) + "_" + str(i) + ".jpg", 'wb') as f:
                    f.write(img_file.content)
                i = i+1
    dettagli.append(dettagli_row)

A Jupyter Widget

In [22]:
ds_dettagli = pd.DataFrame(dettagli)
ds_dettagli.set_index("id")
ds_dettagli.head()

,Comune,Prezzo,Tipologia,date_ad,descrizione,email,id,lat,lon,nome_venditore,phone,userId
0,Brescia (BS),275 €,Cellulari e Smartphone,2019-06-19 10:05:23,Come vendo mio IPhone 7 usato Completo di tut...,,300878677,45.539835,10.219534,Privato,3333636009,101377125
1,Romano di Lombardia (BG),460 €,Cellulari e Smartphone,2019-06-19 10:00:14,"Vendo iPhone 8 rosso, ha una crepa dietro ma l...",,300877611,44.059145,12.563105,Amin,3510802549,102370508
2,Varese (VA),200 €,Cellulari e Smartphone,2019-06-19 09:59:28,Vendo IPhone 6s 16GB Gold perfettamente funzio...,,300877472,45.839713,8.754158,Christian,N/A,2539466
3,Busto Arsizio (VA),350 €,Cellulari e Smartphone,2019-06-19 09:58:24,Vendo iPhone 7 Plus nero con ben 128gb. Perfet...,,300877282,42.549089,-2.242041,Max,N/A,22187676
4,Legnano (MI),229 €,Cellulari e Smartphone,2019-06-19 09:56:04,Nuovo!! con scontrino valido per la garanzia d...,,300876918,45.594699,8.918360,Gianni,3774746058,4561897


In [23]:
ds_dettagli.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 12 columns):
Comune            5 non-null object
Prezzo            5 non-null object
Tipologia         5 non-null object
date_ad           5 non-null object
descrizione       5 non-null object
email             5 non-null object
id                5 non-null object
lat               5 non-null float64
lon               5 non-null float64
nome_venditore    5 non-null object
phone             5 non-null object
userId            5 non-null object
dtypes: float64(2), object(10)
memory usage: 560.0+ bytes


In [24]:
ds_dettagli.to_csv("./subito_it_dettagli.csv")